In [1]:
%load_ext autoreload
%autoreload 2

from final_model import train_and_save

<h2>Notes for Trevor</h2>
If you want to load a new training_test split (or if you change parameters like augmentation or balance), then select False for use_existing_split

Balance now uses augmented images to balance the dataset (so augmentation is ignored)

Check the final_model folder for outputs. If you want to train on the entire training dataset you should first run it on the train/test split (which will save the training params) and THEN run it on the whole thing. You can adjust the full model's training params in final_model/training_params.


Every time you run a model on the train/test split it will save all of the analytical data I think we need, as well as the config settings in model_results.csv

<h2>Set config values</h2>

In [2]:
config = {
    # Files
    'output_folder': './final_model',
    'train_csv': './COMP90086_2024_Project_train/train.csv',
    'train_img_dir': './preprocessed_images/train',
    'test_csv': './final_model/full_predictions.csv',
    'test_img_dir': './preprocessed_images/test',
    'training_params_file': 'training_params.json',

    # Training parameters
    'model': 'EfficientSpatialChannelAttentionNet',
    'target_column': 'stable_height',
    'additional_columns': [], # You'll need to train these first if you want to use
    'balance_dataset': False,
    'use_augmentation': True,
    'use_quantized': False,
    'val_ratio': 0.1,
    'batch_size': 48,
    'dropout_rate': 0.3,
    'learning_rate': 0.001,
    'lr_factor': 0.5,
    'lr_patience': 1,
    'freeze_layers': False,
    'num_epochs': 30,
    'use_existing_split': True, # See above
    'early_stopping_patience': 3,
    'weight_decay': 1e-4 # L2
}

<h2>Train with validation set</h2>

In [3]:
train_and_save(config)

Loading existing split and stats from ./final_model/split_and_stats.json
Model: EfficientSpatialChannelAttentionNet
Target feature: stable_height (6 categories)
Epoch 1/30


Training:   0%|          | 0/576 [00:00<?, ?it/s]

KeyboardInterrupt: 

<h2>Train on whole set and predict</h1>

In [38]:
train_and_save(config, use_full_dataset=True, do_predictions=True)

Loading existing split and stats from ./final_model/split_and_stats_full.json
Model: EfficientAttentionNet
Target feature: stable_height (6 categories)
Epoch 1/5


Training:   0%|          | 0/640 [00:00<?, ?it/s]

Train Loss: 1.2288, Train Acc: 48.87%
Learning Rate: 0.001000
------------------------------------------------------------
Epoch 2/5


Training:   0%|          | 0/640 [00:00<?, ?it/s]

Train Loss: 0.9604, Train Acc: 62.02%
Learning Rate: 0.001000
------------------------------------------------------------
Epoch 3/5


...

Train Loss: 0.8395, Train Acc: 67.28%
Learning Rate: 0.001000
------------------------------------------------------------
Epoch 4/5


Training:   0%|          | 0/640 [00:00<?, ?it/s]

Train Loss: 0.7495, Train Acc: 71.48%
Learning Rate: 0.001000
------------------------------------------------------------
Epoch 5/5


Training:   0%|          | 0/640 [00:00<?, ?it/s]

Train Loss: 0.6762, Train Acc: 74.37%
Learning Rate: 0.001000
------------------------------------------------------------
Predictions saved to ./final_model/full_predictions.csv


In [3]:
for feature in ['shapeset', 'instability_type', 'type', 'cam_angle']:
    config['target_column'] = feature
    train_and_save(config, do_predictions=False)
    train_and_save(config, use_full_dataset=True, do_predictions=True)

Loading existing split and stats from ./final_model/split_and_stats.json
Model: EfficientAttentionNet
Using quantized images: False
Additional features: 
Target feature: shapeset (2 categories)


/workspace/COMP90086_Project/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30


Training:   0%|          | 0/432 [00:00<?, ?it/s]

Validating:   0%|          | 0/12 [00:00<?, ?it/s]

...

Train Loss: 0.0180, Train Acc: 99.38%
Val Loss: 0.0791, Val Acc: 97.79%
Learning Rate: 0.001000
------------------------------------------------------------
Epoch 6/30


Training:   0%|          | 0/432 [00:00<?, ?it/s]

Validating:   0%|          | 0/12 [00:00<?, ?it/s]

Train Loss: 0.0221, Train Acc: 99.22%
Val Loss: 0.1663, Val Acc: 95.96%
Learning Rate: 0.000100
------------------------------------------------------------
Early stopping triggered after 6 epochs
Training parameters saved to training_params.json


TypeError: record_results() missing 1 required positional argument: 'file_path'